# Game of Life


The [Game of Life](http://en.wikipedia.org/wiki/Conway's_Game_of_Life) (GoF) is a cellular automaton devised by the British mathematician John Horton Conway in 1970. The game is a zero-player game, meaning that its evolution is determined by its initial state, requiring no further input. One interacts with the Game of Life by creating an initial configuration and observing how it evolves, or, for advanced players, by creating patterns with particular properties.

The universe of the Game of Life is an infinite two-dimensional orthogonal grid of square cells, each of which is in one of two possible states, live or dead. Every cell interacts with its eight neighbours, which are the cells that are directly horizontally, vertically, or diagonally adjacent. At each step in time, the following transitions occur:

* Any live cell with fewer than two live neighbours dies, as if by underpopulation.
* Any live cell with more than three live neighbours dies, as if by overcrowding.
* Any live cell with two or three live neighbours lives, unchanged, to the next generation.
* Any dead cell with exactly three live neighbours becomes a live cell.

The initial pattern (generation 0) constitutes the "seed" of the system.
Time is considered discrete, and each time step (sometimes called a tick), the previous rules are applied simultaneously to every cell in the grid; "births" and "deaths" are immediate.
Each generation is a pure function of the previous generation alone.


## Assignments

* Begin by implementing the GoF's rules and create few examples with basic seeds in small dimensions. Hint: create `.gif` animations from sequences of still `.png` images
* Implement examples of the three categories of patterns *still lifes*, *oscillators* and *spaceships* (check the [wiki link](http://en.wikipedia.org/wiki/Conway's_Game_of_Life))
* Analyse the evolutions of these patterns in terms of frequency (the number of cycles required to return to the original configuration, if any), occupancy (how many alive cells there are at every step of the game), replication or heat (the average number of cells that change state at each generation), speed (if applicable), etc.
* Increase the size of the GoF's world and play with more advanced and complex patterns

In [1]:
import copy
import numpy as np

from PIL import Image
from IPython.display import HTML

In [2]:
class State:
    """ 
    Represents the state of a Game of Life iteration as
    a set of alive cells. 
    A cell is represented as a (x,y) tuple of coordinates.
    """
    
    def __init__(self, grid_size, initial_state):
        """
        Initialize the state with the grid dimension and
        the starting seed represented as a set of
        alive cells' coordinates.
        """
        self.grid_size = grid_size
        self.state = initial_state
                
    def alive_cells(self):
        """ Returns the set of alive cells. """
        return self.state
            
    def is_alive(self, cell):
        """ Given a cell returns True if the cell is alive. """
        return cell in self.state
    
    def kill(self, cell):
        """ Kills the given cell. """
        self.state.remove(cell)
    
    def revitalize(self, cell):
        """ Revitalizes the given cell if dead. """
        self.state.add(cell)
        
    def neighbours(self, cell):
        """ Returns the 8 neighbouring cells. """
        offsets = [(x,y) for x in range(-1,2) for y in range(-1,2) if not (x == 0 and y == 0)]
        neighbours = set()
        for offset in offsets:
            x, y = cell[0] - offset[0], cell[1] - offset[1]
            if x < self.grid_size[0] and y < self.grid_size[1] and x >= 0 and y >= 0:
                neighbours.add((x,y))
        return neighbours
    
    def count_alive_neighbours(self, cell):
        """ Returns the number of alive neighbours of the given cell. """
        neighbours = self.neighbours(cell)
        count = 0
        for neighbour in neighbours:
            if self.is_alive(neighbour):
                count += 1
        return count
    
    def get_frame(self, size):
        """ Returns the state represented as a grayscale frame of the given size. """
        frame = np.zeros(self.grid_size, dtype=np.uint8)
        for cell in self.alive_cells():
            # cell is (x,y) frame has shape (rows, cols)
            frame[cell[1], cell[0]] = 255
        frame = Image.fromarray(frame)
        frame = frame.resize(size, resample=Image.NEAREST)
        return frame

In [3]:
class Game:
    
    def __init__(self, initial_state, grid_size=(500, 500)):
        self.size = grid_size
        self.state = State(grid_size, initial_state)
        self.history = []
        
    def run(self, iterations):
        """ Run the game for the given number of iterations. """
        
        for _ in range(iterations):
            
            # keep track of the state
            # TODO: render immediately image or save the 'raw' state?
            self.history.append(self.state)
            
            prev_state = self.state
            curr_state = copy.deepcopy(self.state)
            
            for cell in prev_state.alive_cells():
                
                # Any live cell with fewer than two live neighbours dies, as if by underpopulation.
                # Any live cell with more than three live neighbours dies, as if by overcrowding.
                alive_neighbours = prev_state.count_alive_neighbours(cell)
                if alive_neighbours < 2 or alive_neighbours > 3:
                    curr_state.kill(cell)
                # Any live cell with two or three live neighbours lives, unchanged, to the next generation.
                
                # Any dead cell with exactly three live neighbours becomes a live cell.
                neighbours = prev_state.neighbours(cell)
                for neighbour in neighbours:
                    alive_neighbours = prev_state.count_alive_neighbours(neighbour)
                    if alive_neighbours == 3:
                        curr_state.revitalize(neighbour)
                    
            self.state = curr_state        
    
    # https://pillow.readthedocs.io/en/latest/handbook/image-file-formats.html#saving-sequences
    def export_gif(self, filename, size=(512,512)):
        """ Export the game history as an animated gif of the given resolution. """
        frames = []
        for state in self.history:
            frames.append(state.get_frame(size))            
        frames[0].save(filename, save_all=True, append_images=frames[1:], duration=250)

In [4]:
blinker = {(1,0), (1, 1), (1,2)}


game = Game(initial_state=blinker, grid_size=(3, 3))
            
game.run(1000)

filename = 'out.gif'
game.export_gif(filename)

In [5]:
HTML(f'<img src="{filename}">')